In [52]:
PUBLIC_EXPENSIVE_SECRET_API = 'API_KEY'
import os

os.environ["OPENAI_API_KEY"] = PUBLIC_EXPENSIVE_SECRET_API
# print(f"[API KEY]\n{os.environ['OPENAI_API_KEY']}")

from langchain_openai import ChatOpenAI
from langchain_teddynote.messages import stream_response  # 스트리밍 출력
from langchain_core.prompts import PromptTemplate
from langchain.schema import HumanMessage, SystemMessage, Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA



In [34]:
from langchain_community.document_loaders import PyMuPDFLoader
# 단계 1: 문서 로드(Load Documents)
loader = PyMuPDFLoader("./data/[법률_규범_특허][코트라][2022]한-인도 CEPA 활용법 및 인도 통상 애로.pdf")
docs = loader.load()
print(f"문서의 페이지수: {len(docs)}")

문서의 페이지수: 41


In [35]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)
print(f"분할된 청크의수: {len(split_documents)}")

분할된 청크의수: 58


In [36]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings


# BM25를 이용한 Sparce search(keyward 기반) retriever에 docs를 불러오기
bm25_retriever = BM25Retriever.from_documents(
    docs,
)
bm25_retriever.k = 5


# FAISS vector db를 이용한 Semantic search(문맥 vector 유사도 기반) retriever 불러오기

# # 오픈 소스 임베딩 함수를 생성합니다.
# # stf_embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")


# FAISS를 사용하여 문서와 임베딩으로부터 데이터베이스를 생성합니다.
database = FAISS.from_documents(docs, embeddings)

faiss_retriever = database.as_retriever(search_kwargs={"k":5})

In [37]:
# Ensenble retriever 선
ensemble_retriever = EnsembleRetriever(

    retrievers=[bm25_retriever, faiss_retriever],
    weights=[0.65, 0.35], # 가중치
    search_type='mmr', # mmr 기반
)

In [38]:
ensemble_retriever.get_relevant_documents(query)

[Document(page_content='인도 수출 및 CEPA 활용 시 주의사항 알아두기 | 11\n2. 원산지증명서\n가. 원산지증명서 오류 발급 시 정정 방법\n① 주요 내용\n한-인도 CEPA 협정 4.4조에 의하면 수출국에서 선적일로부터 7일 내 원산지증명서를 발급하여야 하며, 7\n일 이후에 발급된 원산지증명서의 경우에는 ‘소급적용(ISSUED RETROSPECTIVELY)’ 문구가 표시되어 있\n어야 한다. 해당 내용이 협정과 다르게 표시될 경우, 원산지증명서 오류 발급으로 간주된다. \n② 사례\n한국에서 인도로 스피커 부품을 수출하고 있는 B사는 기업 측의 착오로 한국 세관에 원산지증명서 발급 신\n청 시 선적일로부터 5일이 되었음에도 ‘소급적용’ 문구 표시를 신청하여 발급받았다. 인도 세관에서는 한-인\n도 CEPA 협정에 따라 해당 원산지증명서를 적합하지 않다고 판단하여, ‘소급적용’ 문구가 없는 원산지증명\n서를 제출할 것을 요청하였다. 이와 같은 오류가 있는 경우, 한-인도 CEPA 적용을 위해 원산지증명서의 재\n발급이 필요하나 재발급을 위해서는 기존에 발급된 원산지증명서 원본을 회수한 이후에 가능하므로 한-인도 \nCEPA 적용이 늦어지게 되고, 통관 지연에 따른 보관료 등이 발생할 것으로 예상되었다. 뉴델리 CEPA활용\n지원센터는 한국 세관과 관련 내용을 확인한 후, 다음과 같은 정정 발급을 통해 보다 빠르게 한-인도 CEPA\n를 적용할 수 있도록 안내하였다.\n1. 한국 세관에\n원산지증명서 정정신청 \n2. 정정 원산지증명서 제출,\n한-인도 CEPA 적용\n3. 원산지증명서\n원본(오류 발급본) 회수\n③ 시사점\n자유무역협정의 이행을 위한 관세법의 특례에 관한 법률 제12조에 따라 관세당국에서 수출물품의 원산지\n증명 능력이 있다고 인정받은 경우, 원산지증명서 발급이 간소화 된다. 해당 인증을 취득한 기업(원산지 인증 \n수출자)은 원산지증명서 발급 신청 시 해당 물품의 원산지를 입증하기 위한 자료 제출이 생략되고, 발

In [39]:
query = "인도 DRI(국세정보국) 관세조사 시 대응방안에 대한 사례를 알려주세요."
ensemble_retriever.invoke(query)

[Document(page_content='08 | 사례로 정리해보는 한-인도 CEPA 활용법 및 인도 통상 애로\n기재하여 신청하면 가능하다. 다만, 한국과 인도의 양국에서 사용하는 HS 코드에 따른 원산지기준이 다를 경\n우, 원산지기준을 모두 충족하여야 한다.\n나. 인도 DRI(국세정보국) 관세조사 시 대응방안\n① 주요 내용\n2021년 3월, 철도 신호용 계전기 관련 인도 대법원의 판결(Westinghouse Saxby FarmerLtd. Vs. \nCommisioner of Central Excise)에 따라 개별 부품이 자동차의 부품으로 분류될 근거가 됨으로써 인도 국\n세정보국(DRI)는 자동차 부품 관련 자의적 해석에 근거한 관세조사를 시행하고 있다. 이로 인해, 인도에 진\n출한 관련 기업들의 한-인도 CEPA  활용 불가 및 차액 관세 납부 등의 피해가 발생하고 있다. \n한-인도 CEPA 적용을 위한 원산지증명서 서식\n자료: 한-인도 CEPA 협정문\n', metadata={'source': './data/[법률_규범_특허][코트라][2022]한-인도 CEPA 활용법 및 인도 통상 애로.pdf', 'file_path': './data/[법률_규범_특허][코트라][2022]한-인도 CEPA 활용법 및 인도 통상 애로.pdf', 'page': 6, 'total_pages': 41, 'format': 'PDF 1.6', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'Adobe InDesign 17.2 (Macintosh)', 'producer': 'Adobe PDF Library 16.0.7', 'creationDate': "D:20220610145706+09'00'", 'modDate': "D:20220614154828+09'00'", 'trapped': ''}),
 Document(page_content='인도 수출 및 CEPA\n활용 시 주의사항 알아두기 \

In [40]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder



# 사용자 질문 맥락화 프롬프트
contextualize_q_system_prompt = """
주요 목표는 사용자의 질문을 이해하기 쉽게 다시 작성하는 것입니다.
사용자의 질문과 채팅 기록이 주어졌을 때, 채팅 기록의 맥락을 참조할 수 있습니다.
채팅 기록이 없더라도 이해할 수 있는 독립적인 질문으로 작성하세요.
질문에 바로 대답하지 말고, 필요하다면 질문을 다시 작성하세요. 그렇지 않다면 질문을 그대로 반환합니다.        
"""
contextualize_q_prompt = ChatPromptTemplate.from_messages([
    ("system", contextualize_q_system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
])

# 질문 프롬프트
qa_system_prompt = """
당신은 질문에 대해 정보를 제공해주는 어시스턴트입니다.
소규모 기업 혹은 스타트업에 인도에 수출 사업을 도와 사실 기반의 정보만을 제공해주면서 가능한 한 도움이 되도록 만들어졌습니다.
사전 정보가 아닌 주어진 자료를 참고하여 정보를 제공해주세요.
만약 자료에 나와있지 않는 상황의 경우에는 인터넷에 검색해보라고 말해주세요.

자료:
{context}

질문:
{input}

FORMAT:
- 진출 사업
- 진출 사업 동향
- 진출 사업 메리트
- 진출 사업 관련 정책
- 진출 사업을 위한 도움말
- 진출 사업 관련 알아야 할 것들
"""

qa_prompt = ChatPromptTemplate.from_messages([
    ("system", qa_system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
])

In [41]:
llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    max_tokens=2048,
    temperature=0,
)

In [42]:
# from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers.string import StrOutputParser
from langchain.chains import create_history_aware_retriever
from langchain.chains.retrieval import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from operator import itemgetter
output_parser = StrOutputParser()


history_aware_retriever = create_history_aware_retriever(
    llm = llm,
    retriever = ensemble_retriever,
    prompt = contextualize_q_prompt
)

# 응답 생성 + 프롬프트 엔지니어링
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)



In [43]:
rag_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
           | EnsembleRetriever(retrievers=[BM25Retriever(vectorizer=<rank_bm25.BM25Okapi object at 0x000001D6A5BEF250>, k=5), VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001D6A5CB88E0>, search_kwargs={'k': 5})], weights=[0.65, 0.35]))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], templa

In [44]:
query = "인도 DRI(국세정보국) 관세조사 시 대응방안에 대한 사례를 알려주세요."
result = rag_chain.invoke(
    {
        "input": query,
        "chat_history": [],
    }
)

In [45]:
result['answer']

'**진출 사업:** 인도 DRI(국세정보국) 관세조사 시 대응방안\n\n**진출 사업 동향:** 인도 DRI(국세정보국)의 관세조사는 자동차 부품 관련 자의적 해석에 근거하여 시행되고 있습니다.\n\n**진출 사업 메리트:** 대비책을 마련하여 관세조사에 대응할 수 있다면, 한-인도 CEPA 활용 불가 및 차액 관세 납부 등의 피해를 최소화할 수 있습니다.\n\n**진출 사업 관련 정책:** 2022년 1월 5일, 인도 관세간접세위원회(CBIC)는 세관 및 국세정보국(DRI)의 자의적 품목분류를 규제하기 위한 지침을 발표하였습니다.\n\n**진출 사업을 위한 도움말:** 관련 기업들은 근거자료를 확보하고, HS 협약에 따른 규정을 우선 적용하며 다양한 사례를 종합적으로 참고하여 판단해야 합니다.\n\n**진출 사업 관련 알아야 할 것들:** 관세 관련 고시 및 지침은 인도 관세간접세위원회(CBIC)의 공식 웹사이트에서 확인할 수 있으며, 관세조사에 대비하는 대응방안을 마련하는 것이 중요합니다.'

In [46]:
query = "원산지증명서 오류 발급 시 정정 방법을 알려줘"
result = rag_chain.invoke(
    {
        "input": query,
        "chat_history": [],
    }
)

In [47]:
result['answer']

"원산지증명서 오류 발급 시 정정하는 방법은 다음과 같습니다:\n\n1. 한-인도 CEPA 협정에 따르면 수출국에서 선적일로부터 7일 이내에 원산지증명서를 발급하여야 합니다. 7일 이후에 발급된 원산지증명서의 경우에는 '소급적용(ISSUED RETROSPECTIVELY)' 문구가 표시되어 있어야 합니다. 만약 이 내용이 협정과 다르게 표시되어 있다면, 원산지증명서 오류로 간주됩니다.\n\n2. 원산지증명서 오류가 발생했을 경우, 정정을 위해 다음 단계를 따를 수 있습니다:\n   - 한국 세관에 원산지증명서 정정신청을 합니다.\n   - 정정된 원산지증명서를 제출하고, 한-인도 CEPA를 적용합니다.\n   - 원산지증명서의 원본(오류 발급본)을 회수합니다.\n\n3. 원산지증명서의 정정을 통해 한-인도 CEPA를 빠르게 적용할 수 있도록 하기 위해, 관련 당국과 협의하여 정확한 절차를 따라야 합니다.\n\n이러한 절차를 통해 원산지증명서 오류를 정정하고, 한-인도 CEPA 협정을 적용할 수 있습니다."

In [48]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory


store = {}  # 세션 기록을 저장할 딕셔너리


def get_session_history(session_ids: str) -> BaseChatMessageHistory:
    print(session_ids)
    if session_ids not in store:  # 세션 ID가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환


with_message_history = (
    RunnableWithMessageHistory(  # RunnableWithMessageHistory 객체 생성
        rag_chain,  # 실행할 Runnable 객체
        get_session_history,  # 세션 기록을 가져오는 함수
        input_messages_key="input",  # 입력 메시지의 키
        history_messages_key="history",  # 기록 메시지의 키
    )
)

In [49]:
query = "원산지증명서 오류 발급 시 정정 방법을 알려줘"
SESSION_ID = "abc123"

result = with_message_history.invoke(
    # 수학 관련 질문 "코사인의 의미는 무엇인가요?"를 입력으로 전달합니다.
    {
        "input": query,
        "chat_history": [],
    },
    # 설정 정보로 세션 ID "abc123"을 전달합니다.
    config={"configurable": {"session_id": SESSION_ID}},
)

abc123


Error in RootListenersTracer.on_chain_end callback: KeyError('output')


In [50]:
result['answer']

'원산지증명서 오류 발급 시 정정 방법은 다음과 같습니다:\n\n1. 한국 세관에 원산지증명서 정정신청을 합니다.\n2. 정정된 원산지증명서를 제출하고, 한-인도 CEPA를 적용합니다.\n3. 원산지증명서의 원본(오류 발급본)을 회수합니다.\n\n이러한 절차를 통해 오류가 있는 원산지증명서를 정정하고, 한-인도 CEPA를 적용할 수 있습니다.원산지증명서의 정정은 신속하게 처리되어야 하며, 관련된 모든 당사자들에게 적절히 통보되어야 합니다.'

In [51]:
query = "인도 DRI(국세정보국) 관세조사 시 대응방안에 대한 사례를 알려주세요."
SESSION_ID = "abc123"

result = with_message_history.invoke(
    # 수학 관련 질문 "코사인의 의미는 무엇인가요?"를 입력으로 전달합니다.
    {
        "input": query,
        "chat_history": [],
    },
    # 설정 정보로 세션 ID "abc123"을 전달합니다.
    config={"configurable": {"session_id": SESSION_ID}},
)

result['answer']

abc123


Error in RootListenersTracer.on_chain_end callback: KeyError('output')


'**진출 사업:** 인도 DRI(국세정보국) 관세조사 시 대응방안\n\n**진출 사업 동향:** 인도 DRI(국세정보국)의 관세조사는 자동차 부품 관련 자의적 해석에 근거한 조사를 시행하고 있으며, 이로 인해 한-인도 CEPA 활용 불가 및 차액 관세 납부 등의 피해가 발생하고 있습니다.\n\n**진출 사업 메리트:** 대비책을 마련하여 인도 DRI(국세정보국)의 관세조사에 대응할 수 있다면, 한-인도 CEPA 혜택을 최대한 활용하고 추가 관세 부담을 최소화할 수 있습니다.\n\n**진출 사업 관련 정책:** 2022년 1월 5일, 인도 관세간접세위원회(CBIC)가 세관 및 국세정보국(DRI)의 자의적 품목분류를 규제하기 위한 지침을 발표하였습니다.\n\n**진출 사업을 위한 도움말:** 관련 기업들은 근거자료를 확보하고, HS 협약에 따른 규정을 우선 적용하며, 다양한 사례를 종합적으로 참고하여 판단하는 것이 중요합니다.\n\n**진출 사업 관련 알아야 할 것들:** 인도 DRI(국세정보국)의 관세조사에 대비하기 위해서는 정확한 원산지증명서를 제출하고, HS 코드에 대한 이해와 관련 서류를 준비하는 것이 중요합니다. 또한, 인도 관세간접세위원회(CBIC)의 공식 웹사이트를 통해 관세 관련 고시 및 지침을 확인하는 것이 도움이 될 수 있습니다.'